In [ ]:
import numpy as np
import statsmodels.tsa.stattools as stm
import scipy.stats as sc
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sb
from sklearn.cluster import KMeans, DBSCAN
import findspark
from pyspark.sql import SparkSession

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

findspark.init()
spark = SparkSession.builder.appName("Price").getOrCreate()

def sql_to_df(command: str) -> pd.DataFrame:
    return spark.sql(command).toPandas()

In [ ]:
spark.stop()